In [ ]:
# INSTRUCTIONS
#
# This script will create a "downloads" folder in the root of the project:
#
# 1) Spruce volume data from LUKE (2015??)
# 2) Aerial orthophotos of Helsinki (2019)
#
#
# BEFORE RUNNING:
#
# Make sure you don't have a previous "downloads" -folder in the project root
#
# RUNNING THE SCRIPT:
#
# Should create the "donwloads" -folder and folders "spruce_volume" and "helsinki2019" in it.
# "spruce_volume" should get filled with spruce volume files from LUKE
# "helsinki2019" should get filler with aerial orthophotos of Helsinki


from owslib.wms import WebMapService
import numpy as np
import os

downloads_path = "../downloads/"
spruce_volume_path = "../downloads/spruce_volume/"
orthophotos_path = "../downloads/helsinki2019/"

if os.path.exists(downloads_path):
    print("Remove the old download-folder from the project in order to continue")
else:
    print("Adding download folder...")
    os.makedirs(downloads_path)
    os.makedirs(spruce_volume_path)
    os.makedirs(orthophotos_path)

    URL = {"luke": "https://kartta.luke.fi/geoserver/MVMI/wms?service=wms&version=1.3.0&request=GetCapabilities", 
           "helsinki": "https://kartta.hel.fi/ws/geoserver/avoindata/wms?request=getCapabilities",
           "syke": "https://paikkatieto.ymparisto.fi/arcgis/services/INSPIRE/SYKE_Maanpeite/MapServer/WMSServer?request=GetCapabilities&service=WMS"}

    OUT_DIR = {"luke": spruce_volume_path,
               "helsinki": orthophotos_path}
    dataset = {"luke": "kuusi_1519",
               "helsinki": "avoindata:Ortoilmakuva_2019_20cm"}

    # boundary coordinates
    x_min = 24.90468
    y_min = 60.18871
    x_max = 24.94942
    y_max = 60.22032

    no_tiles_x = 10 #number of pictures along x-axis
    no_tiles_y = 10 #number of pictures along y-axis

    # the boundaries for small pictures
    xs = np.linspace(x_min, x_max, no_tiles_x + 1)
    ys = np.linspace(y_min, y_max, no_tiles_y + 1) 

    for source in ["luke", "helsinki"]:
        print("")
        print(f"Processing data from source: {source}")
        print("")
        wms = WebMapService(URL[source], version='1.3.0')
        for i in range(0, no_tiles_x):
            print("")
            print(f"Processing column {i}")
            print("")
            for j in range(0, no_tiles_y):
                bbox = (xs[i], ys[j], xs[i+1], ys[j+1])
                img = wms.getmap(layers=[dataset[source]], srs='CRS:84', bbox=bbox, size=(512, 512), format='image/jpeg')
                filename = f"{dataset[source]}_{xs[i]}_{ys[j]}_{xs[i+1]}_{ys[j+1]}.jpg"
                out = open(OUT_DIR[source] + filename, 'wb')
                out.write(img.read())
                print("--------------------")
                print("ADDED FILE : " + filename)
                print("TO FOLDER:   " + OUT_DIR[source])
                print("--------------------")
                out.close()